# AD Classification Notebook

In this notebook we load the **Group 6 model**, which was fine-tuned on the ADReSS dataset.  
The model weights can be downloaded here: [weights](https://technionmail-my.sharepoint.com/my?id=%2Fpersonal%2Fyarden%5Fnahum%5Fcampus%5Ftechnion%5Fac%5Fil%2FDocuments%2F%D7%A4%D7%A8%D7%95%D7%99%D7%A7%D7%98%20%D7%AA%D7%9B%D7%9F&viewid=80aca3e3%2Dbd79%2D43a9%2D91ca%2D05ae2b1505ec)

After loading the model, we use it to classify the synthetic stories we generated for different personas.  
At the end, a **DataFrame** is displayed, showing for each persona the model’s probability estimates (healthy vs. AD) together with its confidence.  

In [1]:
import json
from pathlib import Path
from typing import List, Dict, Union, Optional
import torch
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoTokenizer, AutoModel
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
class ADClassifier(nn.Module):
    def __init__(self, model_name: str, dropout=0.1, hidden_dim=None):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        h = self.bert.config.hidden_size
        h2 = hidden_dim or (h // 2)
        self.drop = nn.Dropout(dropout)
        self.fc1 = nn.Linear(h, h2)
        self.bn = nn.BatchNorm1d(h2)
        self.act = nn.ReLU()
        self.fc2 = nn.Sequential(
            nn.Linear(h2, h2//2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2//2, 2)
        )

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = out.pooler_output
        x = self.drop(x)
        x = self.fc1(x)
        x = self.bn(x)
        x = self.act(x)
        x = self.drop(x)
        return self.fc2(x)

In [ ]:
# -------- CONFIG --------
MODEL_NAME = "bert-base-uncased"
CKPT_PATH = "best_model_base_bert.pt"
MAX_LEN = 512         # max tokens
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------- LOAD MODEL + TOKENIZER --------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = ADClassifier(model_name=MODEL_NAME, dropout=0.1).to(DEVICE)
state = torch.load(CKPT_PATH, map_location=DEVICE)
model.load_state_dict(state)
model.eval()

In [ ]:
@torch.no_grad()
def predict_proba(texts: List[str]) -> torch.Tensor:
    """Return class probabilities [N, 2]: [:,0]=Control, [:,1]=AD."""
    all_probs = []
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i+BATCH_SIZE]
        enc = tokenizer(
            batch_texts,
            truncation=True,
            padding=True,               # dynamic padding
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        enc = {k: v.to(DEVICE) for k, v in enc.items()}
        logits = model(enc["input_ids"], enc["attention_mask"])  # [B, 2]
        probs = torch.softmax(logits, dim=-1).cpu()
        all_probs.append(probs)
    return torch.cat(all_probs, dim=0) if all_probs else torch.empty(0, 2)


def compute_predicted_start_age(ages: List[int], prob_ad: List[float], threshold: float = 0.5) -> Optional[int]:
    """
    Optional helper:
    Return the first age where prob_ad >= threshold (default 0.5).
    If never crosses threshold, return None.
    """
    if not ages:
        return None
    # Ensure ages and prob_ad are aligned and sorted by age
    paired = sorted(zip(ages, prob_ad), key=lambda x: x[0])
    for age, p in paired:
        if p >= threshold:
            return age
    return None

In [ ]:
def df_from_person_json(person: Dict, add_predicted_start_age: bool = False, threshold: float = 0.5) -> pd.DataFrame:
    """
    Build a DataFrame for a single person JSON with 'start_deterioration_age' column preserved.
    """
    name = person.get("name", "UNKNOWN")
    # <-- keep provided start age
    sda = person.get("start_deterioration_age", None)
    stories = person.get("stories", [])

    texts = [s.get("story", "") for s in stories]
    ages = [s.get("age", None) for s in stories]

    probs = predict_proba(texts)  # [N,2]
    if probs.numel() == 0:
        cols = ["person_name", "start_deterioration_age", "age",
                "story", "prob_control", "prob_ad", "pred_label", "confidence"]
        return pd.DataFrame(columns=cols)

    prob_control = probs[:, 0].numpy()
    prob_ad = probs[:, 1].numpy()
    pred_label = probs.argmax(dim=1).numpy()
    confidence = probs.max(dim=1).values.numpy()

    df = pd.DataFrame({
        "person_name": name,
        "start_deterioration_age": sda,            # <-- column per row
        "age": ages,
        "story": texts,
        "prob_control": prob_control,
        "prob_ad": prob_ad,
        "pred_label": pred_label,                  # 0=Control, 1=AD
        "confidence": confidence
    }).sort_values("age", kind="stable").reset_index(drop=True)

    if add_predicted_start_age:
        predicted = compute_predicted_start_age(
            df["age"].tolist(), df["prob_ad"].tolist(), threshold=threshold)
        # same value for all rows of this person
        df["predicted_start_age"] = predicted
        # store threshold as metadata (optional)
        df.attrs["predicted_start_age_threshold"] = threshold

    return df

In [ ]:
def run_on_json_file(json_path: Union[str, Path], add_predicted_start_age: bool = False, threshold: float = 0.5) -> pd.DataFrame:
    with open(json_path, "r", encoding="utf-8") as f:
        person = json.load(f)
    return df_from_person_json(person, add_predicted_start_age=add_predicted_start_age, threshold=threshold)


def run_on_json_dir(json_dir: Union[str, Path], add_predicted_start_age: bool = False, threshold: float = 0.5) -> pd.DataFrame:
    json_dir = Path(json_dir)
    dfs = []
    for p in sorted(json_dir.glob("*.json")):
        try:
            df = run_on_json_file(
                p, add_predicted_start_age=add_predicted_start_age, threshold=threshold)
            df["source_file"] = p.name
            dfs.append(df)
        except Exception as e:
            print(f"[WARN] Failed on {p}: {e}")
    if not dfs:
        cols = ["person_name", "start_deterioration_age", "age", "story",
                "prob_control", "prob_ad", "pred_label", "confidence", "source_file"]
        if add_predicted_start_age:
            cols.append("predicted_start_age")
        return pd.DataFrame(columns=cols)
    return pd.concat(dfs, ignore_index=True)

# Usage

In [ ]:
dementia_data_dir = Path("stories/data_oct_10/dementia/")
healthy_data_dir = Path("stories/data_oct_6/healthyAging/")

In [ ]:

df_one = run_on_json_file(
    f"{dementia_data_dir}/anthony-samuel-reyes.json",
    add_predicted_start_age=True,
    threshold=0.5,
)
df_one

## Prediction on dementia stories

In [ ]:
# Path to your folder with persona JSONs (new dementia stories)
# Run predictions over all personas in the folder
df_all = run_on_json_dir(
    dementia_data_dir, add_predicted_start_age=True, threshold=0.5)

# Display the combined DataFrame
display(df_all)

# --- Informative Plot ---
# Plot prob_ad vs. age for each person
plt.figure(figsize=(10, 6))
for person_name, subdf in df_all.groupby("person_name"):
    plt.plot(subdf["age"], subdf["prob_ad"], marker="o", label=person_name)

plt.axhline(0.5, color="red", linestyle="--", label="Threshold=0.5")
plt.xlabel("Age")
plt.ylabel("Probability of AD")
plt.title(f"AD Probability by Age per Person {dementia_data_dir}")
plt.xticks(range(60, 78, 3))
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

## Healthy stories

In [ ]:

df_one = run_on_json_file(
    f"{healthy_data_dir}/anthony-samuel-reyes.json",
    add_predicted_start_age=True,
    threshold=0.5,
)
df_one

In [ ]:
# Path to your folder with persona JSONs (new dementia stories)
# Run predictions over all personas in the folder
df_all = run_on_json_dir(
    healthy_data_dir, add_predicted_start_age=True, threshold=0.5)

# Display the combined DataFrame
display(df_all)

# --- Informative Plot ---
# Plot prob_ad vs. age for each person
plt.figure(figsize=(10, 6))
for person_name, subdf in df_all.groupby("person_name"):
    plt.plot(subdf["age"], subdf["prob_ad"], marker="o", label=person_name)

plt.axhline(0.5, color="red", linestyle="--", label="Threshold=0.5")
plt.xlabel("Age")
plt.ylabel("Probability of AD")
plt.title(f"AD Probability by Age per Person {healthy_data_dir}")
plt.xticks(range(60, 78, 3))
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()